In [ ]:
# Script to load GEMME scores and variant annotations to build a prediction model

In [1]:
import pandas as pd
from Bio import SeqIO
import numpy as np


In [42]:
# Load the GEMME data
gemme = pd.read_csv("O00555/O00555_normPred_evolCombi.txt", sep=" ", header=0)

print(gemme.shape)
print(gemme.iloc[0:10,0:7])

(20, 2506)
         V1        V2        V3        V4        V5        V6        V7
a -1.902288       NaN -1.801436 -1.775125 -0.184715 -0.715972 -0.320246
c -1.902288 -1.274720 -1.801436 -1.775125 -0.892025 -1.226970 -1.226748
d -1.902288 -1.897429 -1.775027 -1.902387 -0.202129       NaN -0.053653
e -1.902288 -1.897429 -1.775027 -1.902387 -0.143644 -0.069703       NaN
f -1.902288 -1.897429 -1.902288       NaN -1.166047 -1.314861 -1.566667
g -1.902288 -1.897429 -1.902288 -1.902387       NaN -0.687261 -0.486660
h -1.902288 -1.897429 -1.775027 -1.902387 -0.892495 -0.880362 -0.180419
i -1.274720 -1.556523 -1.902288 -1.737903 -1.166047 -1.314861 -1.384619
k -1.902288 -1.897429 -1.274720 -1.902387 -1.126510 -1.270916 -1.566667
l -1.274720 -1.556523 -1.902288 -0.615258 -1.166047 -1.314861 -1.384619


In [43]:
# optional step to review protein-wide stats

# Flatten the dataframe
gemme_combined = gemme.melt(var_name='variable', value_name='value')

# Print summary statistics
print(gemme_combined.describe())
# print(f"Mean={gemme_combined['value'].mean():.2f} SD={gemme_combined['value'].std():.2f}")


              value
count  47614.000000
mean      -1.991261
std        2.191271
min       -9.009544
25%       -3.190148
50%       -1.023401
75%       -0.284953
max        0.581580


In [44]:
# Function to load and process FASTA file
def process_fasta_file(fasta_path):
    # Read the FASTA file
    fasta_sequences = list(SeqIO.parse(fasta_path, "fasta"))
    
    # Assuming only one sequence in the FASTA file
    sequence = str(fasta_sequences[0].seq)
    
    # Split the sequence by amino acids and create a data frame
    sequence_df = pd.DataFrame([list(sequence)])
    
    # Rename the columns to "aminoacid+position"
    sequence_df.columns = [f"{aa}{i+1}" for i, aa in enumerate(sequence)]
    
    return sequence_df


In [45]:
# load a given fasta file and update GEMME scores
sequence_df = process_fasta_file("O00555/O00555.fasta")

# Replace column names with the names from sequence_df
gemme.columns = sequence_df.columns
# Capitalize all row indexes to match mutation letter size
gemme.index = gemme.index.str.upper()

In [46]:
print(gemme.iloc[0:10,0:7])
print(gemme.index[0:10])

         M1        A2        R3        F4        G5        D6        E7
A -1.902288       NaN -1.801436 -1.775125 -0.184715 -0.715972 -0.320246
C -1.902288 -1.274720 -1.801436 -1.775125 -0.892025 -1.226970 -1.226748
D -1.902288 -1.897429 -1.775027 -1.902387 -0.202129       NaN -0.053653
E -1.902288 -1.897429 -1.775027 -1.902387 -0.143644 -0.069703       NaN
F -1.902288 -1.897429 -1.902288       NaN -1.166047 -1.314861 -1.566667
G -1.902288 -1.897429 -1.902288 -1.902387       NaN -0.687261 -0.486660
H -1.902288 -1.897429 -1.775027 -1.902387 -0.892495 -0.880362 -0.180419
I -1.274720 -1.556523 -1.902288 -1.737903 -1.166047 -1.314861 -1.384619
K -1.902288 -1.897429 -1.274720 -1.902387 -1.126510 -1.270916 -1.566667
L -1.274720 -1.556523 -1.902288 -0.615258 -1.166047 -1.314861 -1.384619
Index(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L'], dtype='object')


In [54]:
# Load the variants to diseases annotation file from UniProt/ClinVar
variants = pd.read_csv("O00555/O00555-variants.csv")

# Create a Disease column based on the Phenotype length
variants['Disease'] = np.where(variants['Phenotype'].str.len() > 0, 1, 0)

# Separate missense and other mutations
pattern = r'^[A-Z][0-9]+[A-Z]$'
variants_single = variants[variants['AAChange'].str.match(pattern)].copy()
variants_nonsingle = variants[~variants['AAChange'].str.match(pattern)].copy()



In [55]:
print(len(variants_single))
print(len(variants_nonsingle))

print(variants_single.iloc[0:10,])


2600
255
   Position AAChange                                          Phenotype  \
0         2      A2S  Developmental and epileptic encephalopathy, 42...   
1         2      A2V  Developmental and epileptic encephalopathy, 42...   
2         3      R3C  Developmental and epileptic encephalopathy, 42...   
3         3      R3G  Developmental and epileptic encephalopathy, 42...   
4         5      G5V                                                NaN   
5         8      M8I                                                NaN   
6         9      P9L  Developmental and epileptic encephalopathy, 42...   
7         9      P9S                                                NaN   
8        10     A10S                                                NaN   
9        10     A10V  Developmental and epileptic encephalopathy, 42...   

   Disease  
0        1  
1        1  
2        1  
3        1  
4        0  
5        0  
6        1  
7        0  
8        0  
9        1  


In [56]:
# Function to retrieve a value by column and row name
def get_gemme_score(address):
    # Extract the column name (all except last character) and row name (last character)
    col_name = address[:-1]
    row_name = address[-1]
    
    # Retrieve the value from the dataframe
    return gemme.at[row_name, col_name]

# Add a new column to with retrieved values
variants_single['GEMME'] = variants_single['AAChange'].apply(get_gemme_score)



In [268]:
# Bin the values into two buckets based on Disease value
variants_single_benign = variants_single[variants_single['Disease'] == 0]
variants_single_deleterious = variants_single[variants_single['Disease'] == 1]

print(f"Total variants Benign={variants_single_benign.shape[0]}, Deleterious={variants_single_deleterious.shape[0]}")
print("Of which those with GEMME score <= -2")
print("Benign=",variants_single_benign[variants_single_benign['GEMME'] <= -2].shape[0])
print("Deleterious=",variants_single_deleterious[variants_single_deleterious['GEMME'] <= -2].shape[0])


Total variants Benign=1655, Deleterious=945
Of which those with GEMME score <= -2
Benign= 192
Deleterious= 217


In [ ]:
# Step 1: Generate Input Vectors Based on a Window -5 to +5 Around the Mutation Position

In [211]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef


In [353]:
# Sigmoid function for scaling
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Function to extract window -5 to +5 around the mutation position
def extract_window_scores(row, gemme_df, window_size=5):
    # Extract the position from AAChange (e.g., A123B -> 123)
    position = int(row['AAChange'][1:-1]) - 1  # -1 because positions are 0-indexed in Python
    
    # Define the window range
    start = max(0, position - window_size)
    end = min(gemme_df.shape[1], position + window_size + 1)
    
    # Extract the window of GEMME scores (for all rows)
    window_scores = gemme_df.iloc[:, start:end].values
    
    # Apply the sigmoid function to scale the GEMME values
    window_scores = sigmoid(window_scores)
    
    # Check if the extracted window has the correct size
    if window_scores.shape[1] < 2 * window_size + 1:
        # Calculate necessary padding
        padding_left = max(0, window_size - position)
        padding_right = max(0, window_size - (gemme_df.shape[1] - position - 1))
        
        # Pad with NaN (or another value, if desired)
        window_scores = np.pad(window_scores, ((0, 0), (padding_left, padding_right)), 'constant', constant_values=np.nan)
    
    # Replace NaN values with 1
    window_scores = np.nan_to_num(window_scores, nan=1)
    
    # Flatten the window to create a 1D vector
    window_scores = window_scores.flatten()
    
    # Extract the GEMME score for the mutation itself (before applying sigmoid)
    gemme_score = row['GEMME']
    
    # Apply the sigmoid function to scale the GEMME score
    gemme_score = sigmoid(gemme_score)
    
    # Add the GEMME score for the mutation itself as an additional feature
    window_scores = np.append(window_scores, gemme_score)
    
    return window_scores



In [473]:
# Apply the function to each row in variants_single to create input vectors
ws = 5
vectors = variants_single.apply(lambda row: extract_window_scores(row, gemme, window_size=ws), axis=1)

# Convert the list of window scores into a numpy array
X = np.array(vectors.tolist())
Y = variants_single['Disease'].values


In [474]:
print(X.shape, X[0].shape, X[1000].shape, X[0:5][0:3])
print(Y.shape, Y[0:10])

(2600, 221) (221,) (221,) [[1.         1.         1.         1.         0.1298497  1.
  0.14167638 0.14490613 0.45395204 0.32828065 0.42061578 1.
  1.         1.         1.         0.1298497  0.21845038 0.14167638
  0.14490613 0.29069204 0.22671217 0.2267511  1.         1.
  1.         1.         0.1298497  0.13039974 0.14491835 0.12983856
  0.44963902 1.         0.48658986 1.         1.         1.
  1.         0.1298497  0.13039974 0.14491835 0.12983856 0.46415069
  0.48258132 1.         1.         1.         1.         1.
  0.1298497  0.13039974 0.1298497  1.         0.2375703  0.21167453
  0.17269199 1.         1.         1.         1.         0.1298497
  0.13039974 0.1298497  0.12983856 1.         0.3346426  0.38068077
  1.         1.         1.         1.         0.1298497  0.13039974
  0.14491835 0.12983856 0.29059525 0.29310277 0.45501725 1.
  1.         1.         1.         0.21845038 0.17414613 0.1298497
  0.14957944 0.2375703  0.21167453 0.20026816 1.         1.
  1.        

In [ ]:
# Step 2: Split the Data into Training and Control Subsets

In [475]:
# Split the dataset into training and testing sets with a 70:30 ratio and stratification
ts = 0.35
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, stratify=Y, random_state=123)

# Baseline accuracy: Assign all labels to the dominating class
# Find the most frequent class in the training data
most_frequent_class = np.bincount(Y_train).argmax()

# Create a baseline prediction where all predictions are the most frequent class
Y_baseline_pred = np.full_like(Y_test, most_frequent_class)

# Calculate the baseline accuracy
baseline_accuracy = accuracy_score(Y_test, Y_baseline_pred)
print(f'Baseline Accuracy: {baseline_accuracy:.2f}')


Baseline Accuracy: 0.64


In [235]:
# Step 3: Train the Prediction Model

In [236]:
# ===> SVM

from sklearn.svm import SVC


In [476]:
# Re-train the SVM model using other kernels
# kernels = ['linear', 'poly', 'rbf', 'sigmoid']
kernels = ['rbf']

for kernel in kernels:
    svm_model = SVC(kernel=kernel, probability=True, random_state=123)
    svm_model.fit(X_train, Y_train)
    
    # Evaluate the model
    train_score = svm_model.score(X_train, Y_train)
    test_score = svm_model.score(X_test, Y_test)
    print(f'{kernel.capitalize()} SVM - Training Accuracy: {train_score:.2f}')
    print(f'{kernel.capitalize()} SVM - Test Accuracy: {test_score:.2f}')
    
    # Make predictions on the test set
    Y_pred_test = svm_model.predict(X_test)
    # Calculate Matthews Correlation Coefficient (MCC)
    mcc = matthews_corrcoef(Y_test, Y_pred_test)
    print(f"MCC for Test subset: {mcc:.3f}")
    

Rbf SVM - Training Accuracy: 0.80
Rbf SVM - Test Accuracy: 0.66
MCC for Test subset: 0.189


In [225]:
# ===> XGBoost

import xgboost as xgb


In [477]:
# Initialize the XGBoost Random Forest classifier
xgb_model = xgb.XGBRFClassifier(random_state=123)

# Train the XGBoost Random Forest model
xgb_model.fit(X_train, Y_train)

# Evaluate the model
train_score = xgb_model.score(X_train, Y_train)
test_score = xgb_model.score(X_test, Y_test)
print(f'XGBoost Random Forest - Training Accuracy: {train_score:.2f}')
print(f'XGBoost Random Forest - Test Accuracy: {test_score:.2f}')

# Calculate Matthews Correlation Coefficient (MCC)
Y_pred_test = xgb_model.predict(X_test)
mcc = matthews_corrcoef(Y_test, Y_pred_test)
print(f"XGBoost Random Forest - MCC: {mcc:.3f}")

# Print out the hyperparameters used
print("\nHyperparameters used in the XGBoost Random Forest model:")
for param, value in xgb_model.get_params().items():
    if value is not None:
        print(f'{param}: {value}')
    

XGBoost Random Forest - Training Accuracy: 0.77
XGBoost Random Forest - Test Accuracy: 0.66
XGBoost Random Forest - MCC: 0.202

Hyperparameters used in the XGBoost Random Forest model:
colsample_bynode: 0.8
learning_rate: 1.0
reg_lambda: 1e-05
subsample: 0.8
objective: binary:logistic
enable_categorical: False
missing: nan
random_state: 123


In [202]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, randint


In [203]:
# !!! Skip this cell as it takes a while with no major advantage in results

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300],         # Number of trees in the forest
    'max_depth': [2, 3, 5, 7],                   # Maximum depth of each tree
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 1.0], # Learning rate (step size shrinkage)
    'subsample': [0.6, 0.7, 0.8, 0.9],           # Subsample ratio of the training instances
    'colsample_bynode': [0.6, 0.7, 0.8, 0.9],    # Subsample ratio of columns when constructing each tree
    'reg_lambda': [1e-05, 1e-03, 1e-01]          # L2 regularization term on weights
}


# Initialize the XGBoost Random Forest classifier
xgb_model = xgb.XGBRFClassifier(random_state=123)

# Set up the grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train, Y_train)

# Best parameters found
print(f"Best parameters found: {grid_search.best_params_}")

# Best score achieved with the best parameters
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")

# Evaluate on the test set
test_score = grid_search.score(X_test, Y_test)
print(f"Test Accuracy with best parameters: {test_score:.2f}")


Fitting 5 folds for each of 3840 candidates, totalling 19200 fits
Best parameters found: {'colsample_bynode': 0.6, 'learning_rate': 1.0, 'max_depth': 5, 'n_estimators': 50, 'reg_lambda': 0.1, 'subsample': 0.7}
Best cross-validation accuracy: 0.66
Test Accuracy with best parameters: 0.67


In [227]:
# ===> Ridge Regression

from sklearn.linear_model import RidgeClassifier


In [478]:
# Initialize the Ridge classifier
ridge_model = RidgeClassifier(random_state=123)

# Train the Ridge model
ridge_model.fit(X_train, Y_train)

# Evaluate the model
train_score = ridge_model.score(X_train, Y_train)
test_score = ridge_model.score(X_test, Y_test)
print(f'Ridge Regression - Training Accuracy: {train_score:.2f}')
print(f'Ridge Regression - Test Accuracy: {test_score:.2f}')

# Calculate Matthews Correlation Coefficient (MCC)
Y_pred_test = ridge_model.predict(X_test)
mcc = matthews_corrcoef(Y_test, Y_pred_test)
print(f"Ridge Regression - MCC: {mcc:.3f}")


Ridge Regression - Training Accuracy: 0.72
Ridge Regression - Test Accuracy: 0.64
Ridge Regression - MCC: 0.158


In [229]:
# ===> LASSO Regression

from sklearn.linear_model import LogisticRegression


In [479]:
# Initialize the Lasso (L1-regularized Logistic Regression) classifier
lasso_model = LogisticRegression(penalty='l1', solver='liblinear', random_state=123)

# Train the Lasso model
lasso_model.fit(X_train, Y_train)

# Evaluate the model
train_score = lasso_model.score(X_train, Y_train)
test_score = lasso_model.score(X_test, Y_test)
print(f'Lasso Regression - Training Accuracy: {train_score:.2f}')
print(f'Lasso Regression - Test Accuracy: {test_score:.2f}')

# Calculate Matthews Correlation Coefficient (MCC)
Y_pred_test = lasso_model.predict(X_test)
mcc = matthews_corrcoef(Y_test, Y_pred_test)
print(f"Lasso Regression - MCC: {mcc:.3f}")


Lasso Regression - Training Accuracy: 0.72
Lasso Regression - Test Accuracy: 0.65
Lasso Regression - MCC: 0.173
